In [1]:
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn import metrics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("../windows_datasets_FJSMC/datasets/DS utilizados/MHEALTHDATASET_S10_98.csv")
data.drop(["Sensor", "ID_MHE"], axis=1, inplace=True)

subjects = data["Subject"]

data.drop("Subject", axis=1, inplace=True)

y = data["Activity_Number"]

data.drop("Activity_Number", axis=1, inplace=True)

x = data[data.columns.to_list()]

x.head()

,Acctr_tChest_X(m/s2),Acctr_tChest_Y(m/s2),Acctr_tChest_Z(m/s2),L1_tChest(Mv),L2_tChest(Mv),Acctr_tLA_X(m/s2),Acctr_tLA_Y(m/s2),Acctr_tLA_Z(m/s2),Gype_tLA_X(deg/s),Gype_tLA_Y(deg/s),...,Acctr_tRLA_Z(m/s2),Gype_tRLA_X(deg/s),Gype_tRLA_Y(deg/s),Gype_tRLA_Z(deg/s),MgFld_tRLA_X(uT),MgFld_tRLA_X(T),MgFld_tRLA_Y(uT),MgFld_tRLA_Y(T),MgFld_tRLA_Z(uT),MgFld_tRLA_Z(T)
0,-9.4945,-1.03400,-1.04040,-0.079540,-0.12977,0.26331,-9.6396,1.3852,0.70501,-0.47842,...,3.9951,0.66275,-0.43326,0.66595,0.54755,5.475500e-07,0.89477,8.947700e-07,-0.720020,-7.200200e-07
1,-9.5761,-0.99217,-1.26820,-0.079540,-0.12977,0.36777,-9.7072,1.5600,0.72356,-0.48218,...,4.1479,0.66275,-0.43326,0.66595,0.72351,7.235100e-07,0.53732,5.373200e-07,0.009097,9.096900e-09
2,-9.6850,-1.10510,-1.19420,-0.092098,-0.12140,0.48946,-9.6479,1.5433,0.72356,-0.48218,...,4.0750,0.66275,-0.43326,0.66595,0.54577,5.457700e-07,0.71692,7.169200e-07,-0.357260,-3.572600e-07
3,-9.3680,-0.79141,-0.93438,-0.092098,-0.12977,0.27516,-9.6480,1.5412,0.69388,-0.48593,...,4.1434,0.66667,-0.44353,0.66810,0.54392,5.439200e-07,0.53186,5.318600e-07,-0.716460,-7.164600e-07
4,-9.5541,-1.08310,-1.24480,-0.075353,-0.11722,0.33479,-9.5896,1.4405,0.69388,-0.48593,...,4.0124,0.66667,-0.44353,0.66810,0.72539,7.253900e-07,0.72599,7.259900e-07,0.729270,7.292700e-07


In [3]:
x = (x-x.min(axis = 0))/(x.max(axis = 0) - x.min(axis = 0))

x = np.array(x)

In [ ]:
# Configuración de hiperparámetros
C_values = np.logspace(-3, 3, 13)  # Rango de valores de C en escala logarítmica

n_repeats = 20
k_folds = 10

svm_acc = np.zeros(len(C_values))

for i, C in enumerate(C_values):
    print("C =", C)

    cv = RepeatedKFold(n_splits=k_folds, n_repeats=n_repeats)

    # Crear modelo SVM con kernel RBF
    model = SVC(kernel='rbf', C=C, gamma=0.001)

    # Evaluar el modelo utilizando validación cruzada
    acc = cross_val_score(model, x, y, scoring=metrics.make_scorer(metrics.accuracy_score), cv=cv, n_jobs=-1)

    # Calcular precisión media
    svm_acc[i] = acc.mean()

# Encontrar el valor de C que maximiza la precisión media
best_C_index = np.argmax(svm_acc)
best_C = C_values[best_C_index]

print("Mejor valor de C:", best_C)



In [5]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

model = SVC(C=100,kernel="rbf",gamma=0.001)
model.fit(X_train, y_train)

y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)

print(model.score(X_test, y_test))
print(metrics.accuracy_score(y_test, y_test_hat))
print(metrics.confusion_matrix(y_test, y_test_hat))
print(metrics.f1_score(y_test, y_test_hat, average=None))
print(metrics.classification_report(y_test, y_test_hat))

0.8141592920353983
0.8141592920353983
[[5831   96   63   61   18    1   46   59   40  123   66   61    0]
 [   6  298    0    0    0    0    0    0    0    0    0    0    0]
 [ 221    0  103    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0  323    0    0    0    0    0    0    0    0    0]
 [ 181    0    0    0   90    0    0    0    0    0    0    0    0]
 [ 292    0    0    0    6   10    0    0   13    0    0    0    0]
 [ 101    0    0    0    0    0  137    0    0    0    0    0    0]
 [  70    0    0    0    0    0    0  210    0    0    0    0    0]
 [ 134    0    0    0    0    0    0    0  145    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0  328    0    0    0]
 [  52    0    0    0    0    0    0    0    0    0  266    0    0]
 [  12    0    0    0    0    0    0    0    0    0    0  263    0]
 [ 105    0    0    0    0    0    0    0    0    0    0    0    0]]
[0.8657758  0.85386819 0.42040816 0.91371994 0.46753247 0.06024096
 0.6508313

C:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn.svm import SVC

C_range = np.logspace(-2, 10, 13)
gamma_range = np.logspace(-9, 3, 13)
param_grid = dict(gamma=gamma_range, C=C_range)
cv = StratifiedShuffleSplit(n_splits=10, test_size=0.1)
grid = GridSearchCV(SVC(), param_grid=param_grid, cv=cv, n_jobs=-1)
grid.fit(x, y)

print(
    "The best parameters are %s with a score of %0.2f"
    % (grid.best_params_, grid.best_score_)
)